# Calculate Cross Proxy Perfomance

In [ ]:
from dotenv import load_dotenv
import os
import pandas as pd
load_dotenv()
import pickle as pk
import numpy as np
from pathlib import Path
base_dir = os.getenv('BASEDIR')

from tqdm import tqdm
tqdm.pandas()

# from lightgbm import LGBMClassifier
from flaml.default import LGBMClassifier
from itertools import chain, combinations

In [ ]:
import pandas as pd
import os 
import re
import numpy as np
from tqdm import tqdm

from sklearn.model_selection import cross_val_predict, StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.datasets import make_classification

import os
from collections import defaultdict

In [ ]:
dataset='qanda'
datasets = [dataset]

In [ ]:
fr_groundtruths =['URLa_FR', 'URLb_FR']
lr_groundtruths =['HASHTAG','URL_LR','POLITICIAN_1H_LR', 'PARTY_FOLLOWER_LR']

In [ ]:
performance = {dataset : defaultdict(dict) for dataset in datasets}
gt_f1 = {dataset : defaultdict(dict) for dataset in datasets}
overlap_f1 = {dataset : defaultdict(dict) for dataset in datasets}

In [ ]:
import itertools
def all_combinations(l):
    return(itertools.product(l, repeat=2))

In [ ]:
def load_vector(fp):
    with open(fp, 'rb') as rf:
        vec = pk.load(rf)
    return vec

In [ ]:
granularity = '_per_user'
feature = 'use'

for dataset in datasets:
    for train, test in itertools.chain(all_combinations(lr_groundtruths),all_combinations(fr_groundtruths)):
        train_path = os.path.join(base_dir,'data','03_processed',dataset,'ground_truth', dataset+'_'+train+'_'+granularity+'.pk')
        if not os.path.exists(train_path):
            continue
        test_path = os.path.join(base_dir,'data','03_processed',dataset,'ground_truth', dataset+'_'+test+'_'+granularity+'.pk')
        if not os.path.exists(test_path):
            continue

        y_train = load_vector(train_path).values
        mask_train = y_train >= 0

        y_test = load_vector(test_path).values
        mask_test = y_test >= 0
        mask = mask_train | mask_test
        
        if any(mask):
            # gt_f1[dataset][train][test] = f1_score(y_train[mask], y_test[mask],average='macro')

            # y = y_train[mask]
            y = y_train[mask_train]
            
            feature_path = os.path.join(base_dir,'data','03_processed',dataset,'features', dataset+'_'+feature+'_'+granularity+'.pk')
            X_orig = load_vector(feature_path)
            
            # X = X_orig[mask]
            X = X_orig[mask_train]
            et = LGBMClassifier(is_unbalance=True,n_estimators=200, n_jobs=-1, verbose=-1, seed=123)
            if train == test:
                # et = LGBMClassifier(colsample_bytree=0.8, class_weight='balanced', n_jobs=-1)
                # validation instance
                skf = StratifiedKFold(n_splits=5, shuffle=True)
                # getting the model validation predictions
                preds = cross_val_predict(et, X, y, cv=skf, method='predict_proba')
            else:
                # et = LGBMClassifier(colsample_bytree=0.8, class_weight='balanced', n_jobs=-1)
                et.fit(X,y)
                X_text = X = X_orig[mask_test]
                preds = et.predict_proba(X_text)
            if train in fr_groundtruths:
                preds = preds[:,1]
            # evaluating the model
            try:
                if test == 'POLITICIAN_LR' and train == 'POLITICIAN_1H_LR':
                    N = preds.shape[0]
                    preds = np.c_[np.zeros(N),preds]
                print('Area under the ROC Curve:', roc_auc_score(y_test[mask_test], preds, multi_class='ovo'))
                performance[dataset][train][test] = roc_auc_score(y_test[mask_test], preds, multi_class='ovo')
            except Exception as e:
                print('Failed for:', train, test)
                raise(e)

In [ ]:
col_order = ['HASHTAG','URL_LR','POLITICIAN_1H_LR','PARTY_FOLLOWER_LR','USER_FR','URLa_FR','URLb_FR']
pd.DataFrame(dict(performance[dataset])).reindex(index=col_order, columns=col_order)

In [ ]:
col_order = ['HASHTAG','URL_LR','POLITICIAN_1H_LR','PARTY_FOLLOWER_LR','URLa_FR','URLb_FR']
performance_dfs = {dataset: pd.DataFrame(dict(performance[dataset])).reindex(index=col_order, columns=col_order) for dataset in datasets}

In [ ]:
performance_dfs['qanda']

In [ ]:
performance_dfs['qanda'].to_csv(os.path.join(base_dir, 'data', '04_results', 'qanda_cross_gt_auc.csv'))